In [1]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator

In [2]:
from tensorflow.python.keras.datasets import mnist

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1)
x_test  = x_test.reshape(-1, 28, 28, 1)

# Normalize
x_train = x_train/255.
x_test  = x_test/255.

In [3]:
def make_masking_noised_data(data_x, percent=0.1):
    size = data_x.shape
    masking = np.random.binomial(n=1, p=percent, size=size)
    return data_x * masking

x_train_masked = make_masking_noised_data(x_train)
x_test_masked  = make_masking_noised_data(x_test)

In [4]:
def make_gaussian_noised_data(data_x, scale=0.8):
    gaussed_x = data_x + np.random.normal(loc=0, scale=scale, size=data_x.shape)
    gaussed_x = np.clip(gaussed_x, 0, 1)
    return gaussed_x

x_train_gauss = make_gaussian_noised_data(x_train)
x_test_gauss  = make_gaussian_noised_data(x_test)

In [6]:
from IPython.display import display_png

display_png(array_to_img(x_train[0]))
display_png(array_to_img(x_train_masked[0]))
display_png(array_to_img(x_train_gauss[0]))

# Creating Convolutional Autoencoder Model

In [10]:
cae = Sequential()

# Encoder
cae.add(Conv2D(16, (3, 3), 1, activation='relu', padding='same', input_shape=(28, 28, 1)))
cae.add(MaxPooling2D((2, 2), padding='same'))
cae.add(Conv2D(8, (3, 3), 1, activation='relu', padding='same'))
cae.add(MaxPooling2D((2, 2), padding='same'))

# Decoder
cae.add(Conv2D(8, (3, 3), 1, activation='relu', padding='same'))
cae.add(UpSampling2D((2, 2)))
cae.add(Conv2D(16, (3, 3), 1, activation='relu', padding='same'))
cae.add(UpSampling2D((2, 2)))
cae.add(Conv2D(1, (3, 3), 1, activation='sigmoid', padding='same'))

cae.compile(optimizer='adam', loss='binary_crossentropy')

initial_weights = cae.get_weights()

cae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 7, 7, 8)           584       
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 14, 14, 16)        1168      
__________